⚠️ 이 노트북은 서울 건물연식지도 데이터 전처리 과정을 공유하기 위해서 작성되었습니다. 리포에 원본 데이터를 커밋하지 않았으므로 바로 실행되지 않습니다. 

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.colors as mc
import os
from lonboard.geoarrow.geopandas_interop import geopandas_to_geoarrow
import pyarrow as pa


### Read Building data 2017/2024

In [ ]:
b_2023_origin = gpd.read_file('./F_FAC_BUILDING_서울.zip', engine="pyogrio", encoding='euc-kr')

In [ ]:
b_2023_origin.shape

In [ ]:
b_2023 = b_2023_origin.copy()
b_2023 = b_2023.to_crs(4326)

#### Filter out outliars manually

In [ ]:
b_2023 = b_2023[b_2023.UFID !='2020202526764540434800000000']
b_2023 = b_2023[b_2023.UFID !='2019202463904539575400000000']

#### Give arbiturary value to the buildings with unlikely HEIGHT value

In [ ]:
b_2023.loc[b_2023['HEIGHT'] > 10000, 'HEIGHT'] = 5

In [ ]:
b_2017_origin = gpd.read_file('./building_2017.geojson', engine="pyogrio")
b_2017 = b_2017_origin.copy()

In [ ]:
b_2023_origin[b_2023_origin.UFID =='2020202526764540434800000000'].area

#### Read helper data

In [ ]:
dong_code = pd.read_csv('./dc.txt', delimiter='\t')

### Slim down 2023 data - GeoJSON - pmtiles (Prerequisite: Tippecanoe )

### Assign 법정동 to buildings

In [ ]:
dong_code['bd_code'] = dong_code['법정동코드'].astype(str).str[:10]
b_2023['bd_code'] = b_2023['BD_MGT_SN'].astype(str).str[:10]
# bw_dong = b_2023.join(dong_code, lsuffix='l', rsuffix='r')
bw_dong = pd.merge(b_2023, dong_code, on='bd_code', how='left')
bw_dong['BEONJI'] = b_2023['BD_MGT_SN'].str[11:19]
bw_dong['DONG'] = bw_dong['법정동명']

bw_dong['APR_Y'] = bw_dong['USEAPR_DAY'].str[:4]
bw_dong['APR_Y'] = bw_dong['APR_Y'].astype(float)

bw_dong_s = bw_dong[['DONG','BEONJI','APR_Y', 'USEAPR_DAY', 'HEIGHT', 'BLD_NM', 'geometry']]

In [ ]:
# Check if the data was processed as expected: 석탄회관 - 수송동 80-6
bw_dong_s[bw_dong_s.BLD_NM =='석탄회관']

#### Centroids

In [ ]:
bc_2023 = b_2023.copy()
centroids_2023 = b_2023['geometry'].centroid
bc_2023['geometry'] = centroids_2023

bc_2017 = b_2017.copy()
centroids_2017 = b_2017['geometry'].centroid
bc_2017['geometry'] = centroids_2017

#### Get rid of null data from centroids

In [ ]:
bc_2017 = bc_2017[bc_2017['USEAPR_DAY'] != '0']
bc_2023 = bc_2023[~bc_2023['USEAPR_DAY'].isna()]

### Write processed files

In [ ]:
file_path_2023 = './output/bd_2023.geojson'
file_path_2017 = './building_2017.geojson'

file_path_2023c = './output/bdc_2023.geojson'
file_path_2017c = './output/bdc_2017.geojson'

bw_dong_s.to_file(file_path_2023)
bc_2017.to_file(file_path_2017c)
bc_2023.to_file(file_path_2023c)

#### Group with 법정동

In [ ]:
# File from https://www.vworld.kr/dtmk/dtmk_ntads_s002.do?dsId=30603
dong_origin = gpd.read_file('./LSMD_ADM_SECT_UMD_서울.zip', engine='pyogrio', encoding='euc-kr') 
dong = dong_origin[['EMD_CD','EMD_NM', 'geometry']]
dong = dong.to_crs('4326')


In [ ]:
b_2023['APR_Y'] = b_2023['USEAPR_DAY'].str[:4]
b_2023['APR_Y'] = b_2023['APR_Y'].astype(float)

In [ ]:
joined_2023 = dong.sjoin(b_2023)
jg_2023 = joined_2023.groupby(['EMD_CD']).agg({'APR_Y': 'mean'})
jg_dong = gpd.GeoDataFrame(dong.merge(jg_2023.reset_index()))

In [ ]:
jg_dong.to_file('./output/dong_2023.geojson')

In [ ]:
joined_2017 = dong.sjoin(b_2017)
joined_2017['Year'] = joined_2017['Year'].replace(0, None)
jg_2017 = joined_2017.groupby(['EMD_CD'], dropna=False).agg({'Year': 'mean'})
jg_dong_2017 = gpd.GeoDataFrame(dong.merge(jg_2017.reset_index()))


# Merge the aggregated results back to the original dong GeoDataFrame
# Assuming dong has a column EMD_CD to match on
jg_dong_2017 = gpd.GeoDataFrame(dong.merge(jg_2017, on='EMD_CD', how='left'))

In [ ]:
jg_dong_2017.to_file('./output/dong_2017.geojson')

#### Generate pmtiles

In [ ]:
command = "tippecanoe -z 16 -Z 14 -d 16 -l bd_2023 -o ./output/bd_2023.pmtiles --drop-densest-as-needed --extend-zooms-if-still-dropping --maximum-zoom=16 " + file_path_2023
os.system(command)

In [ ]:
command = "tippecanoe -z 16 -Z 14 -d 16 -l bd_2017 -o ./output/bd_2017.pmtiles --drop-densest-as-needed --extend-zooms-if-still-dropping --maximum-zoom=16 " + file_path_2017
os.system(command)

##### PMtiles - Centroid data

In [ ]:
command = "tippecanoe --drop-rate=1 --no-line-simplification --no-feature-limit --no-tile-size-limit  -z 14 -Z 10 -l bdc_2023 -o ./output/bdc_2023.pmtiles " + file_path_2023c
#" --drop-densest-as-needed --extend-zooms-if-still-dropping " + file_path_2023c
os.system(command)

In [ ]:
command = "tippecanoe --drop-rate=1 --no-line-simplification --no-feature-limit --no-tile-size-limit -z 14 -Z 10 -l bdc_2017 -o ./output/bdc_2017.pmtiles --drop-densest-as-needed --extend-zooms-if-still-dropping " + file_path_2017c
os.system(command)

### Bonus: Generate GeoParquet

In [ ]:
gpq_b_2023 = b_2023.copy()

#### Color helper

In [ ]:
viridis =  ["#440154", "#482475", "#414287", "#355e8d", "#2a768e", "#218e8d", "#21a585", "#3dbc74", "#70cf57", "#b0dd2f"]
viridis.reverse()
rgb_colors = []
for h in viridis:
  rgb_colors.append([int(m *255) for m in mc.to_rgba(h)]) # rgba / rgb matters?
rgb_colors

In [ ]:
def get_color(row):
  year = row['APR_Y']
  if ((np.isnan(year)) or (year < 1850)):
    return [100,100,100,255]
  cidx = int((year - 1930)/10)
  if cidx > 9:
    cidx = 9
  if cidx < 0:
    cidx = 0;
  return rgb_colors[cidx]

color_cols = gpq_b_2023.apply(get_color, axis=1)

In [ ]:
# Convert the color_column to NumPy array
#gpq_b_2023['color_column'] = gpq_b_2023['color'].apply(np.array, dtype=np.uint8)

color_array = np.stack(color_cols.values) # necessary?

In [ ]:
s_arrow = geopandas_to_geoarrow(gpq_b_2023, preserve_index=False)
s_arrow = s_arrow.append_column(
    "color", pa.FixedSizeListArray.from_arrays(color_array.flatten('C'), 4)
)


In [ ]:
pa.parquet.write_table(s_arrow, './output/bd_2023.parquet')